<a href="https://colab.research.google.com/github/ivoryRabbit/RecSys/blob/master/2_MBCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memory-Based Collaborative Filtering

In [1]:
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from typing import Callable, Tuple, List

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

from tensorflow.keras.utils import get_file
import zipfile

In [2]:
def load_data(data_size : str) -> pd.DataFrame:
    ''' load Movie Lens data '''

    if data_size == '1m':
        fname = 'ml-1m.zip'
        data = 'ml-1m/ratings.dat'
    elif data_size == '10m':
        fname = 'ml-10m.zip'
        data = 'ml-10M100K/ratings.dat'
    elif data_size == '20m':
        fname = 'ml-20m.zip'
        data = 'ml-20m/ratings.csv'
    elif data_size == '25m':
        fname = 'ml-25m.zip'
        data = 'ml-25m/ratings.csv'
    if not glob.glob(data):
        origin = f'http://files.grouplens.org/datasets/movielens/{fname}'
        file = get_file(fname, origin)
        zip_ref = zipfile.ZipFile(file, 'r')
        zip_ref.extractall()

    col_names = ['user_id', 'movie_id', 'rating', 'timestamp']
    if data_size in ['20m', '25m']:
        ratings = pd.read_csv(data, names = col_names, engine = 'python')
    else:
        ratings = pd.read_csv(data, sep = '|', delimiter = '::', names = col_names, engine = 'python')
    print(ratings.shape)
    return ratings

In [3]:
ratings = load_data('1m')
ratings.head()

5922816/5917549 [==============================] - 1s 0us/step
(1000209, 4)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
print(f'# of users = {ratings.user_id.nunique()}')
print(f'# of items = {ratings.movie_id.nunique()}')

# of users = 6040
# of items = 3706


In [5]:
def binarizer(df: pd.DataFrame, threshold = 4) -> pd.DataFrame:
    df = df.assign(rating = np.where(df.rating >= threshold, 1, 0))
    return df[df.rating > 0.0].reset_index(drop = True)

def make_warm(df: pd.DataFrame, threshold = 5) -> pd.DataFrame: # remove cold starters
    positive = df.groupby('user_id')['movie_id'].count()
    positive = positive.index[positive >= threshold]
    return df[df.user_id.isin(positive)].reset_index(drop = True)

def train_valid_test_split(df: pd.DataFrame, size: float) -> pd.DataFrame:
    train_user, test_user = train_test_split(df.user_id.unique(), test_size = 2 * size, random_state = 777)
    valid_user, test_user = train_test_split(test_user, test_size = 0.5, random_state = 777)
    train, valid, test = map(lambda x: df[df.user_id.isin(x)], (train_user, valid_user, test_user))
    train, valid, test = map(lambda df: df.reset_index(drop = True), (train, valid, test))
    return train, valid, test

def query_relev_split(df: pd.DataFrame, size: float) -> pd.DataFrame:
    timeorder = df.groupby('user_id')['timestamp'].rank(method = 'first', ascending = True)
    seen_cnts = df.groupby('user_id')['movie_id'].transform('count')
    df = df.assign(seen_cnts = seen_cnts, timeorder = timeorder)
    query = df[df.timeorder < df.seen_cnts * (1-size)]
    relev = df[df.timeorder >= df.seen_cnts * (1-size)]
    relev = relev[relev.user_id.isin(query.user_id.unique())]
    query, relev = map(lambda df: df.drop(columns = ['timeorder', 'seen_cnts']), (query, relev))
    query, relev = map(lambda df: df.reset_index(drop = True), (query, relev))
    return query, relev

def list_agg(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby('user_id', as_index = False)[['movie_id']].agg(list)

In [6]:
data = binarizer(ratings)
data = make_warm(data)
data.head()

,user_id,movie_id,rating,timestamp
0,1,1193,1,978300760
1,1,3408,1,978300275
2,1,2355,1,978824291
3,1,1287,1,978302039
4,1,2804,1,978300719


In [7]:
train, _, test = train_valid_test_split(data, size = 0.1)
test_q, test_r = query_relev_split(test, size = 0.2)

In [8]:
class MBCF:
    def fit(self, df: pd.DataFrame):
        self.user_idx2id = df.user_id.unique() # list[index] = id
        self.user_id2idx = {e: i for i, e in enumerate(self.user_idx2id)} # dict[id] = index
        self.item_idx2id = df.movie_id.unique() # list[index] = id
        self.item_id2idx = {e: i for i, e in enumerate(self.item_idx2id)} # dict[id] = index
        self.n_user = len(self.user_idx2id)
        self.n_item = len(self.item_idx2id)

        df = df.assign(user_id = df.user_id.map(self.user_id2idx), 
                       movie_id = df.movie_id.map(self.item_id2idx))

        row = df.user_id
        col = df.movie_id
        val = np.ones(shape = df.index.size)
        shape = (self.n_user, self.n_item)
        
        self.mat = csr_matrix((val, (row, col)), shape = shape, dtype = np.float32)
    
    def predict(self, df:pd.DataFrame, N: int, similarity = 'dot') -> pd.DataFrame:
        df = df.assign(movie_id = df.movie_id.map(self.item_id2idx)).dropna()
        df = df.assign(movie_id = df.movie_id.astype(int))
        df = list_agg(df)
        profile = df['movie_id']
        
        pred = []
        for idx in tqdm(df.index):
            query = np.zeros(shape = (self.n_item, ), dtype = np.float32)
            query[profile[idx]] = 1.0

            sim = self.mat.dot(query)
            a = self.mat.dot(np.ones(shape = (self.n_item, ), dtype = np.float32))
            b = np.sum(query)
            if similarity == 'cosine':
                sim /= np.sqrt(a * b)
            elif similarity == 'jaccard':
                sim /= (a + b - sim)
            elif similarity == 'euclidean':
                sim = np.sqrt(a + b - 2 * sim)
            pred_batch = self.mat.T.dot(sim)

            rec = np.argsort(np.where(query == 1.0, -np.inf, pred_batch))[:-N-1:-1]
            rec = self.item_idx2id[rec]
            pred.append({'user_id': df.at[idx, 'user_id'],
                         'movie_id': rec})
        return pd.DataFrame(pred)

In [9]:
class evaluate:
    def __init__(self, true: pd.DataFrame, pred: pd.DataFrame):
        self.true = true
        self.pred = pred
        self.max_K = 10000
        self.idcg = np.cumsum([1.0 / np.log2(i+2) for i in range(self.max_K)])

    def _recall(self, gt: List, rec: List, K = None) -> float: # Recall
        res = [r for r in rec[:K] if r in gt]
        return len(res) / np.min([K, len(gt)])
    
    def _precision(self, gt: List, rec: List, K = None) -> float: # Precision
        res = [r for r in rec[:K] if r in gt]
        return len(res) / len(rec[:K])

    def _AP(self, gt: List, rec: List, K = None) -> float: # Average Precision
        res = 0.0
        for i, r in enumerate(rec[:K]):
            if r in gt:
                res += self._precision(gt, rec[:K], i+1)
        return res / np.min([K, len(gt)])

    def _HR(self, gt: List, rec: List, K = None) -> float: # Hit Rate
        for i, r in enumerate(rec[:K]):
            if r in gt:
                return  1.0
        return 0

    def _RR(self, gt: List, rec: List, K = None) -> float: # Reciprocal Rank
        for i, r in enumerate(rec[:K]):
            if r in gt:
                return  1.0 / (i+1)
        return 0

    def _nDCG(self, gt: List, rec: List, K = None) -> float: # normalized Discounted Cumulative Gain
        dcg = 0.0
        for i, r in enumerate(rec[:K]):
            if r in gt:
                dcg += 1.0 / np.log2(i+2)
        idcg = self.idcg[min([len(gt), K])-1]
        return dcg / idcg

    def _get_item(self, rec: List, K = None) -> float:
        for r in rec[:K]:
            self.uniq_item[r] = self.uniq_item.get(r, 0) + 1

    def __call__(self, K = None):
        self.K = K if K else self.max_K
        self.recall = 0.0
        self.precision = 0.0
        self.MAP = 0.0
        self.HR = 0.0
        self.MRR = 0.0
        self.nDCG = 0.0
        self.uniq_item = {}
        n = self.true.index.size
        for gt, rec in zip(tqdm(self.true.movie_id), self.pred.movie_id):
            self.recall += self._recall(gt, rec, K) / n
            self.precision += self._precision(gt, rec, K) / n
            self.MAP += self._AP(gt, rec, K) / n
            self.HR += self._HR(gt, rec, K) / n
            self.MRR += self._RR(gt, rec, K) / n
            self.nDCG += self._nDCG(gt, rec, K) / n
            self._get_item(rec, K)
        self.CO = len(self.uniq_item) / (n*K) # Coverage
        self.ED = -np.sum([self.uniq_item[i] * np.log(self.uniq_item[i] / (n*K)) for i in self.uniq_item]) / (n*K) # Entropy-Diversity

    def print_all(self):
        K = '@' + str(self.K) if self.K else ''
        print(f'{"Recall":>12}{K} : {self.recall:.5f}',
              f'\n{"Precision":>12}{K} : {self.precision:.5f}',
              f'\n{"MAP":>12}{K} : {self.MAP:.5f}',
              f'\n{"HR":>12}{K} : {self.HR:.5f}', 
              f'\n{"MRR":>12}{K} : {self.MRR:.5f}',
              f'\n{"nDCG":>12}{K} : {self.nDCG:.5f}',
              f'\n{"CO":>12}{K} : {self.CO:.5f}',
              f'\n{"ED":>12}{K} : {self.ED:.5f}')

### Remark

- Since A and B are binary vectors, it satisfies $ A ^2 = A $ by the element-wise product. 

- Futher, note that the dot product "$\cdot$" satisfies that $ A \cdot B = \sum AB $.

In [10]:
model = MBCF()
model.fit(train)

## 1. Dot Product Similarity

$$ \text{DotSim}(A, B) := A \cdot B$$

In [11]:
true = list_agg(test_r)
true.head(5)

,user_id,movie_id
0,3,"[1394, 104, 1079, 1259, 2355, 3552, 1304, 2081]"
1,9,"[2268, 1682, 2278, 590, 524, 529, 2294, 1921, ..."
2,42,"[3421, 3868, 1257, 2997, 2134, 1265, 593, 2143..."
3,79,"[3044, 3176, 2686, 2024]"
4,92,"[2054, 2134, 733, 592, 2072, 3107, 2162, 1291,..."


In [12]:
pred = model.predict(test_q, N = 100)
pred.head(5)

,user_id,movie_id
0,3,"[2571, 2028, 589, 593, 1270, 110, 2762, 318, 1..."
1,9,"[1196, 260, 1198, 296, 858, 589, 110, 1197, 23..."
2,42,"[2571, 2028, 2858, 110, 593, 2762, 858, 1580, ..."
3,79,"[2858, 260, 1196, 593, 2762, 2571, 608, 2028, ..."
4,92,"[1196, 260, 2858, 1198, 2028, 1210, 589, 527, ..."


In [13]:
scores = evaluate(true, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.11487 
   Precision@20 : 0.07127 
         MAP@20 : 0.03673 
          HR@20 : 0.55960 
         MRR@20 : 0.20011 
        nDCG@20 : 0.09943 
          CO@20 : 0.00720 
          ED@20 : 3.76680



      Recall@100 : 0.28338 
   Precision@100 : 0.04657 
         MAP@100 : 0.04474 
          HR@100 : 0.83775 
         MRR@100 : 0.20728 
        nDCG@100 : 0.16515 
          CO@100 : 0.00462 
          ED@100 : 5.08964


## 2. Cosine Similarity

$$
\begin{aligned}
\text{CosSim}(A, B) &:= \cos(\angle AB) = \frac{A \cdot B}{|A||B|}
\\ &= \frac{A \cdot B}{\sqrt{\sum A}\sqrt{\sum B}}
\end{aligned}
$$

In [14]:
pred = model.predict(test_q, N = 100, similarity = 'cosine')
pred.head(5)

,user_id,movie_id
0,3,"[2571, 2028, 589, 110, 593, 1270, 2762, 318, 1..."
1,9,"[260, 1196, 1198, 296, 110, 589, 2396, 858, 29..."
2,42,"[2571, 2028, 2858, 110, 1580, 2762, 593, 858, ..."
3,79,"[2858, 260, 1196, 593, 2762, 2571, 2028, 608, ..."
4,92,"[2858, 260, 1196, 1198, 2028, 1210, 589, 527, ..."


In [15]:
scores = evaluate(true, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.11650 
   Precision@20 : 0.07185 
         MAP@20 : 0.03737 
          HR@20 : 0.56291 
         MRR@20 : 0.20064 
        nDCG@20 : 0.10082 
          CO@20 : 0.00745 
          ED@20 : 3.77759



      Recall@100 : 0.29048 
   Precision@100 : 0.04675 
         MAP@100 : 0.04561 
          HR@100 : 0.85265 
         MRR@100 : 0.20820 
        nDCG@100 : 0.16806 
          CO@100 : 0.00475 
          ED@100 : 5.11232


## 3. Jaccard Similarity

$$ \text{JacSim}(A, B) := \frac{|set(A) \cap set(B)|}{|set(A) \cup set(B)|} = \frac{A \cdot B}{\sum A + \sum B - A \cdot B}$$

In [16]:
pred = model.predict(test_q, N = 100, similarity = 'jaccard')
pred.head(5)

,user_id,movie_id
0,3,"[2571, 2028, 589, 110, 1270, 593, 2762, 1240, ..."
1,9,"[260, 1196, 1198, 296, 110, 2396, 589, 2997, 8..."
2,42,"[2571, 2028, 110, 2858, 1580, 2762, 593, 858, ..."
3,79,"[2858, 260, 2762, 1196, 593, 2571, 2028, 608, ..."
4,92,"[2858, 260, 1196, 1198, 2028, 1210, 589, 527, ..."


In [17]:
scores = evaluate(true, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.11777 
   Precision@20 : 0.07268 
         MAP@20 : 0.03847 
          HR@20 : 0.56291 
         MRR@20 : 0.20319 
        nDCG@20 : 0.10246 
          CO@20 : 0.00770 
          ED@20 : 3.79589



      Recall@100 : 0.30000 
   Precision@100 : 0.04765 
         MAP@100 : 0.04705 
          HR@100 : 0.86921 
         MRR@100 : 0.21121 
        nDCG@100 : 0.17237 
          CO@100 : 0.00512 
          ED@100 : 5.14248


## 4. Euclidean Distance

$$
\begin{aligned}
\text{Dist}(A, B) &:= \sqrt{\sum(A - B)^2}
\\ &= \sqrt{\sum (A^2 - 2AB + B^2)}
\\ &= \sqrt{\sum (A + B - 2AB)}
\\ &= \sqrt{\sum A + \sum B - 2 A \cdot B}
\end{aligned}
$$

In [18]:
pred = model.predict(test_q, N = 100, similarity = 'euclidean')
pred.head(5)

,user_id,movie_id
0,3,"[593, 2028, 2762, 2571, 608, 318, 527, 858, 58..."
1,9,"[260, 1196, 1198, 858, 589, 1270, 110, 1197, 2..."
2,42,"[2858, 593, 2028, 2762, 608, 2571, 318, 527, 8..."
3,79,"[2858, 260, 1196, 1198, 593, 2028, 2571, 2762,..."
4,92,"[2858, 260, 1196, 1198, 2028, 608, 1210, 527, ..."


In [19]:
scores = evaluate(true, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.10278 
   Precision@20 : 0.06482 
         MAP@20 : 0.03122 
          HR@20 : 0.52318 
         MRR@20 : 0.17556 
        nDCG@20 : 0.08800 
          CO@20 : 0.00654 
          ED@20 : 3.68328



      Recall@100 : 0.25710 
   Precision@100 : 0.04265 
         MAP@100 : 0.03810 
          HR@100 : 0.80132 
         MRR@100 : 0.18274 
        nDCG@100 : 0.14787 
          CO@100 : 0.00429 
          ED@100 : 4.99696
